In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [2]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Get data

In [3]:
data_loader = dl.data_loader()

In [4]:
abbrs = data_loader.get_ensembles()
data = data_loader.get_fit_data()
mss = {abbr : data[abbr]['mss'] for abbr in sorted(abbrs)}
mk = {abbr : data[abbr]['mk'] for abbr in sorted(abbrs)}
mpi = {abbr : data[abbr]['mpi'] for abbr in sorted(abbrs)}

to_gvar = lambda arr : gv.gvar(arr[0], arr[1])
aw0 = {abbr : to_gvar(data[abbr]['aw0']) for abbr in sorted(abbrs)}

### Make plot

In [5]:
def plot_mss(mss, mk, mpi, aw0):
    
    abbrs = sorted(aw0.keys())
    bs_N = len(mss[abbrs[0]])
    plot_data = {
        0 : {abbr : mss[abbr]**2 for abbr in abbrs},
        1 : {abbr : 2*mk[abbr]**2 - mpi[abbr]**2 for abbr in abbrs}
    }
    color_data = {abbr : np.repeat(aw0[abbr], bs_N) for abbr in abbrs}
    
    # Color by lattice spacing/length
    cmap = matplotlib.cm.get_cmap('rainbow')
    min_max = lambda x : [np.min(x), np.max(x)]
    minimum, maximum = min_max(np.concatenate([gv.mean(color_data[abbr]) for abbr in abbrs]))
    norm = matplotlib.colors.Normalize(vmin=minimum, vmax=maximum)
    
    # Get scatter plot & color data
    
    x = np.zeros(bs_N * len(abbrs))
    y = np.zeros(bs_N * len(abbrs))
    z = np.zeros(bs_N * len(abbrs))
    for j, abbr in enumerate(abbrs):
        x[j*bs_N:(j+1)*bs_N] = gv.mean(plot_data[0][abbr])
        y[j*bs_N:(j+1)*bs_N] = gv.mean(plot_data[1][abbr])
        z[j*bs_N:(j+1)*bs_N] = gv.mean(color_data[abbr])

    # Plot data
    sc = plt.scatter(x, y, c=z, vmin=minimum, vmax=maximum,
                     cmap=cmap, rasterized=True, marker="o")
    
    # Plot labels
    plt.grid()
    plt.xlabel('$m_{ss}^2$', fontsize = 24)
    plt.ylabel('$2m_{K}^2 - m_\pi^2$', fontsize = 24)

    # Format colorbar
    color_bar = plt.colorbar(sc)
    color_bar.set_alpha(0.8)
    color_bar.draw_all()
    color_bar.set_label('$a/w_0$', fontsize = 24)
    
    # Set xlim, ylim -- only works if xy_parameters[i] is a vector, not a scalar
    min_max = lambda x : [np.min(x), np.max(x)]
    xmin, xmax = min_max(np.concatenate([gv.mean(plot_data[0][abbr]) for abbr in abbrs]))
    ymin, ymax = min_max(np.concatenate([gv.mean(plot_data[1][abbr]) for abbr in abbrs]))
    xdelta = xmax - xmin
    ydelta = ymax - ymin
    plt.xlim(xmin-0.05*xdelta, xmax+0.05*xdelta) #xmin-0.05*xdelta
    plt.ylim(ymin-0.05*ydelta, ymax+0.05*ydelta)

    fig = plt.gcf()
    plt.close()
    return fig
    
fig = plot_mss(mss, mk, mpi, aw0)
data_loader.save_plots(fig, output_filename='mss_vs_mk_mpi')

Done.
